В мой текст на русском языке я включила почти все сложные случаи, описанные в статье О.Н. Ляшевской, в частности: омоформы (ели), общепринятые сокращения (тыс., м.), аббревиатуры (МГУ, МФТИ), фамилии на -ий/-ов (их можно спутать с прилагательными), авторские неологизмы (потеннисить) и сложные слова, разделенные дефисом. Все эти слова неоднозачны в трактовке части речи и считаются непростыми для частеречных таггеров: для работы с ними необходимо обращаться к контексту либо внутреннему устройству слова.

In [126]:
text = 'Около двух тыс. студентов и выпускников МГУ и МФТИ, среди них такие известные ученые, как Антон Иванович Петров, Леонид Иосифович Певчих и Алексей Ефимович Лидский, примут участие в соревнованиях по лаун-теннису. Некоторые выпускники-ученые приедут прямиком из Нью-Йорка, Абу-Даби и других городов и стран, чтобы потеннисить на славу. Данные участники должны будут выслать свои персональные данные заранее по электронной почте. Теннисный корт, длина которого достигает десяти м., а ширина – двадцати пяти м., будут окружать специально посаженные ели и березы, которые, как предполагается, поднимут боевой дух участников, напомнив им о родине-матушке. Об этом событии напишут в различных газетах и журналах: выйдут тексты по-русски, по-французски и по-немецки.'
exclude = [',','.', '!', '?', ':', '–', '(', ')']
exclude = set(exclude)
text = ''.join(ch for ch in text if ch not in exclude)
text_pos = '''Около ADP
двух NUM
тыс NUM
студентов NOUN
и CONJ
выпускников NOUN
МГУ NOUN
и CONJ
МФТИ NOUN
среди ADP
них PRON
такие DET
известные ADJ
ученые NOUN
как CONJ
Антон PROPN
Иванович PROPN
Петров PROPN
Леонид PROPN
Иосифович PROPN
Певчих PROPN
и CONJ
Алексей PROPN
Ефимович PROPN
Лидский PROPN
примут VERB
участие NOUN
в ADP
соревнованиях NOUN
по ADP
лаун-теннису NOUN
Некоторые PROPN
выпускники-ученые NOUN
приедут VERB
прямиком ADV
из ADP
Нью-Йорка PROPN
Абу-Даби PROPN
и CONJ
других PROPN
городов NOUN
и CONJ
стран NOUN
чтобы CONJ
потеннисить VERB
на ADP
славу NOUN
Данные ADJ
участники NOUN
должны ADJ
будут AUX
выслать VERB
свои DET
персональные ADJ
данные NOUN
заранее ADV
по ADP
электронной ADJ
почте NOUN
Теннисный ADJ
корт NOUN
длина NOUN
которого PRON
достигает VERB
десяти NUM
м NOUN
а CONJ
ширина NOUN
двадцати NUM
пяти NUM
м NOUN
будут AUX
окружат VERB
специально ADV
посаженные VERB
ели NOUN
и CONJ
березы NOUN
которые PROPN
как CONJ
предполагается VERB
поднимут VERB
боевой ADJ
дух NOUN
участников NOUN
напомнив VERB
им PRON
о ADP
родине-матушке NOUN
Об ADP
этом DET
событии NOUN
напишут VERB
в ADP
различных ADJ
газетах NOUN
и CONJ
журналах NOUN
выйдут VERB
тексты NOUN
по-русски ADV
по-французски ADV
и CONJ
по-немецки ADV''' 
pos = []
for i in text_pos.split('\n'):
    pos.append(i.split()[1])
gold = pos.copy()

Эти две функции приводят теги трех использовавшихся парсеров к моему набору золотого стандарта.

In [127]:
def other_to_udp(pos, type_p):
    for i in range(len(pos)):
        if type_p == 'natasha':
            if pos[i] == 'AUX':
                pos[i] = 'VERB'
            if pos[i] == 'PROPN':
                pos[i] = 'NOUN'
            if pos[i] == 'NPRO' or pos[i] == 'DET':
                pos[i] = 'PRON'
            if pos[i] == 'SCONJ' or pos[i] == 'CCONJ':
                pos[i] = 'CONJ'
        if type_p == 'mystem':
            if pos[i] == 'V':
                pos[i] = 'VERB'
            if pos[i] == 'S':
                pos[i] = 'NOUN'
            if pos[i] == 'A':
                pos[i] = 'ADJ'
            if pos[i] == 'ADVPRO':
                pos[i] = 'ADV'
            if pos[i] == 'ADVPRO':
                pos[i] = 'ADV'
            if pos[i] == 'APRO' or pos[i] == 'SPRO':
                pos[i] = 'PRON'
    return pos

In [128]:
def py_to_udp(pos, type_p):
    pymorphy = []
    for i in range(len(pos)):
        if pos[i] == 'ADJF' or pos[i] =='ADJS':
            pymorphy.append('ADJ')
        elif pos[i] == 'ADVB':
            pymorphy.append('ADV')
        elif pos[i] == 'NUMR':
            pymorphy.append('NUM')
        elif pos[i] == 'INFN' or pos[i] == 'GRND' or pos[i] == 'PRTF':
            pymorphy.append('VERB')
        elif pos[i] == 'PREP':
            pymorphy.append('ADP')
        elif pos[i] == 'NPRO':
            pymorphy.append('PRON')
        else:
            pymorphy.append(str(pos[i]))
    return pymorphy

Анализируем текст с помощью пайморфи.

In [129]:
import pymorphy2
import re
morph = pymorphy2.MorphAnalyzer()
pm2 = []
for word in text.split():
    p = morph.parse(word)[0]
    pm2.append(p.tag.POS)

Анализируем текст с помощью майстема.

In [130]:
from pymystem3 import Mystem
m = Mystem()
mystem = []
for word in text.split():
    an = m.analyze(word)[0].get('analysis')[0].get('gr')
    pos = re.split(r'=|\,', an)[0]
    mystem.append(pos)

Анализируем текст с помощью Наташи.

In [131]:
from natasha import (MorphVocab, NewsMorphTagger, NewsEmbedding, Doc, Segmenter)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
segmenter = Segmenter()
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
natasha = [_.pos for _ in doc.tokens]

Используем функцию для приведения тегов к единому стандарту.

In [132]:
natasha_new = other_to_udp(natasha, 'natasha')
pm_new = py_to_udp(pm2, 'py')
mystem_new = other_to_udp(mystem, 'mystem')

Эта функция считает accuracy.

In [133]:
def get_acc(pos, gold):
    count = 0
    for i in range(len(pos)):
        if pos[i] == gold[i]:
            count+=1
    acc = count/len(pos)
    return acc

Последовательно считаем accuracy каждого парсера.

In [134]:
print("natasha's accuracy:", get_acc(natasha_new, gold))
print("mystem's accuracy:", get_acc(mystem_new, gold))
print("pymorphy's accuracy:", get_acc(pm_new, gold))

natasha's accuracy: 0.8076923076923077
mystem's accuracy: 0.6634615384615384
pymorphy's accuracy: 0.7692307692307693


In [291]:
from nltk import word_tokenize 
from nltk.util import ngrams
import re

Эта функция выделяет синтаксические группы. Я выбрала группы прилагательное+существительное (поскольку отзывы чаще всего описательны, в них много экспрессивных адъективных конструкций, выражающих отношение автора к фильму), не+глагол (я заметила, что в отзывах на кинопоиске люди часто высказывают свое мнение в формате "не зашло", "не может никому понравиться" и так далее, поэтому такие конструкции предположительно будут маркерами отрицательных отзывов) и наречие+глагол/глагол+наречие (чтобы выделять описательные, но глагольные конструкции типа "потрясающе снято", "сделано качественно" и тд). Я использовала пайморфи, хотя Наташа работает лучше, тк с Наташей сложнее работать.

In [ ]:
def get_groups(text):
    adj_noun = []
    no = []
    adv_p = []
    text = ''.join(ch for ch in text if ch not in exclude)
    i = re.split(r'\?|!|\.', text)
    for line in i:
        token = nltk.word_tokenize(line)
        bigram = list(ngrams(token, 2)) 
        for i in bigram:
            if morph.parse(i[0])[0].tag.POS == 'ADJF' and morph.parse(i[1])[0].tag.POS == 'NOUN':
                adj_noun.append(i)
            if i[0] == 'не' and morph.parse(i[1])[0].tag.POS == 'VERB':
                no.append(i)
            if morph.parse(i[0])[0].tag.POS == 'ADVB' and morph.parse(i[1])[0].tag.POS == 'VERB' or morph.parse(i[0])[0].tag.POS == 'VERB' and morph.parse(i[1])[0].tag.POS == 'ADVB':
                adv_p.append(i)
    return adj_noun, no, adv_p

Имплементацию этого кода в старую домашку см. в соседней тетрадке.

In [136]:
#import sys
#!{sys.executable} -m spacy download en_core_web_sm

В тексте на английском языке я использовала в основном омоформы, то есть слова,  которые могут принадлежать к разным частям речи и так же как и в русском требуют контекстадля работыс ними. В их числе были глаголы-существительные (protest), местоимения-союзы (whatever), предлоги-частицы (to), существительные-прилагательные (right).

In [73]:
eng = '''I am going to protest for a right to name this place whatever I want. You need to rate every rain you had in your life. Give me a sign if you want to smoke some stuff with me. She is a real tease. Use some time to vote! Wake up and help me find my watch! He gave me a wave and a wink. His voice is so calm it makes me yawn. I heard his voice and looked upstage. I want to take a risk and participate in the race for a gold flower. Did you burn the cake?'''
exclude = [',','.', '!', '?', ':', '–', '(', ')']
exclude = set(exclude)
eng1 = ''.join(ch for ch in eng if ch not in exclude)
eng_tag = '''I PRON
am AUX
going VERB
to PREP
protest NOUN
for PREP
a DET
right NOUN
to PREP
name NOUN
this DET
place NOUN
whatever PRON
I PRON
want VERB
You PRON
need VERB
to PREP
rate VERB
every DET
rain NOUN
you PRON
had VERB
in PREP
your PRON
life NOUN
Give VERB
me PRON
a DET
sign NOUN
if CONJ
you PRON
want VERB
to PART
smoke VERB
some DET
stuff NOUN
with PREP
me PRON
She PRON
is VERB
a DET
real ADJ
tease NOUN
Use VERB
some DET
time NOUN
to PART
vote VERB
Wake VERB
up PREP
and CONJ
help VERB
me PRON
find VERB
my PRON
watch NOUN
He PRON
gave VERB
me PRON
a DET
wave NOUN
and CONJ
a DET
wink NOUN
His PRON
voice NOUN
is VERB
so ADV
calm ADJ
it PRON
makes VERB
me PRON
yawn VERB
I PRON
heard VERB
his PRON
voice NOUN
and CONJ
looked VERB
upstage ADJ
I PRON
want VERB
to PART
take VERB
a DET
risk NOUN
and CONJ
participate VERB
in PREP
the CONJ
race NOUN
for PREP
a DET
gold ADJ
flower NOUN
Did AUX
you PRON
burn VERB
the DET
cake NOUN'''
pos_eng = []
for i in eng_tag.split('\n'):
    pos_eng.append(i.split()[1])
gold_eng = pos_eng.copy()

Обработаем предложения с помощью Spacy

In [137]:
import spacy
spacy.load('en_core_web_sm')
nlp = spacy.load("en_core_web_sm")
doc = nlp(eng1)
spacy_tags = []
for i, s in enumerate(doc.sents):
    for t in s:
        spacy_tags.append(t.pos_)

In [138]:
print(len(spacy_tags))

101


Обработаем предложения с помощью Flair

In [139]:
import re
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('pos')
t = []
for sentence in re.split(r'\?|!|\.', eng):
    sentence1 = Sentence(sentence)
    tagger.predict(sentence1)
    tags = sentence1.to_tagged_string()
    tags = tags.split()[1::2]
    for i in tags:
        t.append(re.sub(r'>|<', '', i))
    tags_flair = t
print(len(tags_flair))

2020-10-19 23:25:08,052 loading file /Users/apple/.flair/models/en-pos-ontonotes-v0.5.pt
2020-10-19 23:25:13,493 Warning: An empty Sentence was created! Are there empty strings in your dataset?
101


Обработаем предложения с помощью NLTK

In [140]:
from nltk import pos_tag, word_tokenize
sents = re.split(r'\?|!|\.', eng)
[word_tokenize(sent) for sent in sents]
toks = []
for sent in sents:
    toks.append(pos_tag(word_tokenize(sent)))
tags_nltk = []
for s in toks:
    if s != []:
        for w in s:
            tags_nltk.append(w[1])

Приведем все к формату золотого стандарта

In [141]:
def other_to_gold(pos, type_p):
    for i in range(len(pos)):
        if type_p == 'nltk':
            if pos[i] == 'VBZ' or pos[i] == 'VB' or pos[i] == 'VBP' or pos[i] == 'VBD' or pos[i] == 'VBG':
                pos[i] = 'VERB'
            if pos[i] == 'NNP' or pos[i] == 'NN':
                pos[i] = 'NOUN'
            if pos[i] == 'WDT' or pos[i] == 'DT':
                pos[i] = 'DET'
            if pos[i] == 'PRP' or pos[i] == 'PRP$':
                pos[i] = 'PRON'
            if pos[i] == 'RP' or pos[i] == 'TO':
                pos[i] = 'PART'
            if pos[i] == 'JJ':
                pos[i] = 'ADJ'
            if pos[i] == 'RB':
                pos[i] = 'ADV'
            if pos[i] == 'CC':
                pos[i] = 'CONJ'
            if pos[i] == 'IN':
                pos[i] = 'PREP'
        if type_p == 'flair':
            if pos[i] == 'DT':
                pos[i] = 'DET'
            if pos[i] == 'VBZ' or pos[i] == 'VB' or pos[i] == 'VBP' or pos[i] == 'VBD' or pos[i] == 'VBG':
                pos[i] = 'VERB'
            if pos[i] == 'NNP' or pos[i] == 'NN':
                pos[i] = 'NOUN'
            if pos[i] == 'WDT' or pos[i] == 'DT':
                pos[i] = 'DET'
            if pos[i] == 'PRP' or pos[i] == 'PRP$':
                pos[i] = 'PRON'
            if pos[i] == 'RP' or pos[i] == 'TO':
                pos[i] = 'PART'
            if pos[i] == 'JJ':
                pos[i] = 'ADJ'
            if pos[i] == 'RB':
                pos[i] = 'ADV'
            if pos[i] == 'CC':
                pos[i] = 'CONJ'
            if pos[i] == 'IN':
                pos[i] = 'PREP'
        if type_p == 'spacy':
            if pos[i] == 'PROPN':
                pos[i] = 'NOUN'
            if pos[i] == 'CCONJ' or pos[i] == 'SCONJ':
                pos[i] = 'CONJ'
    return pos

In [116]:
spacy_gold = other_to_gold(spacy_tags, 'spacy')
nltk_gold = other_to_gold(tags_nltk, 'nltk')
flair_gold = other_to_gold(tags_flair, 'nltk')

Посчитаем accuracy каждого таггера

In [142]:
print("spacy's accuracy:", get_acc(spacy_gold, gold_eng))
print("nltk's accuracy:", get_acc(nltk_gold, gold_eng))
print("flair's accuracy:", get_acc(flair_gold, gold_eng))

spacy's accuracy: 0.7623762376237624
nltk's accuracy: 0.8613861386138614
flair's accuracy: 0.8811881188118812
